## Great Barrier Reef - Training Script

Downloads the specified branch of the repo from the github and can run the training or even a full parameter sweep.

In [21]:
#@title Select your options
mode = "train"    #@param ["train", "evaluate"]
branch_name = "GBR_starfish_detection"    #@param {type:"string"}
wandb_model = "stuttgartteam8/protect_gbr/2pjrq56n"    #@param ["New", "stuttgartteam8/protect_gbr/2pjrq56n"]
full_dataset = True    #@param {type:"boolean"}

In [2]:
!nvidia-smi

Wed Dec 29 20:56:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# download dataset
#%%capture
%cd /content/
# images 0-478 (part of whole data set)
if full_dataset:
    !gdown https://drive.google.com/uc?id=1DW2CJja9n8d0Bub-jmJZvnKXAHrpYLkv
    !unzip -u "/content/GBR_dataset.zip" -d "/content/";
else:
    !gdown https://drive.google.com/uc?id=1nVROGC31eVwThW7xYUM8zuxlmQVgC9xr
    !unzip -u "/content/GBR_dataset_partial.zip" -d "/content/";

In [28]:
# delete existing repo to ensure the newest is fetched from github 
% cd /
!rm -r /content/dl-lab-21w-team08

# download repo
# %%capture
%cd /content/
git_access_token = "28fdd19659f4a0e5bcaf16dd820e5c68bed91102"

# !git clone https://28fdd19659f4a0e5bcaf16dd820e5c68bed91102@github.tik.uni-stuttgart.de/iss/dl-lab-21w-team08.git
clone_comand = f"git clone --single-branch --branch {branch_name} https://{git_access_token}@github.tik.uni-stuttgart.de/iss/dl-lab-21w-team08.git"
!$clone_comand

/
/content
Cloning into 'dl-lab-21w-team08'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 377 (delta 64), reused 53 (delta 28), pack-reused 287
Receiving objects: 100% (377/377), 6.37 MiB | 5.37 MiB/s, done.
Resolving deltas: 100% (233/233), done.


In [6]:
# install dependencies
%%capture
!pip install wandb
!pip install tensorflow_addons
#!pip install gin-config

In [29]:
%%capture out
# run training script
# 37262d20054e8dbf092705158103cd02e31691d6
%cd /content/
!mkdir wandb
%cd /content/dl-lab-21w-team08/GBR_starfish_detection/

!wandb login 37262d20054e8dbf092705158103cd02e31691d6

In [ ]:
# training

if mode == "train":
    # pass dataset path and the wandb model which is used as starting point for training
    if full_dataset:
        !python main.py -p="/content/GBR_dataset/" --wandb_model=wandb_model
    else:
        !python main.py -p="/content/GBR_dataset_partial/" --wandb_model=wandb_model

In [35]:
# model evaluation

%cd /content/dl-lab-21w-team08/GBR_starfish_detection/
import wandb
import tensorflow as tf
from input import load, annotate_image

wandb.init(project="protect_gbr", entity="stuttgartteam8", mode="disabled") 
if full_dataset:
    arg_update = {"data_dir": "/content/GBR_dataset/"}
else:
    arg_update = {"data_dir": "/content/GBR_dataset_partial/"}
wandb.config.update(arg_update, allow_val_change=True)
config = wandb.config


ds_train, ds_val, ds_test = load(config)


if mode == "evaluate":
    # download the model from wandb
    print("Download model:")
    api = wandb.Api()
    run = api.run("stuttgartteam8/protect_gbr/1khez299")
    run.file("model.h5").download(replace=True)

    print("Load model:")
    model = tf.keras.models.load_model('model.h5', compile=False) 
    print(model.summary())

else:
    # use the model from this training session and evaluate it
    model = tf.keras.models.load_model('model.h5', compile=False) # todo: the path for the model needs to be adapted!

/content/dl-lab-21w-team08/GBR_starfish_detection
   video_id  sequence  video_frame  sequence_frame image_id annotations
0         0     40258            0               0      0-0          []
1         0     40258            1               1      0-1          []
2         0     40258            2               2      0-2          []
3         0     40258            3               3      0-3          []
4         0     40258            4               4      0-4          []
Download model:
Load model:
Model: "yolo"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 448, 448, 3)]     0         
                                                                 
 tf.cast (TFOpLambda)        (None, 448, 448, 3)       0         
                                                                 
 tf.__operators__.getitem (S  (None, 448, 448, 3)      0         
 licingOpLambd

In [ ]:

import tensorflow.keras as keras
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import cv2

for x,y in ds_train:
    y_pred = model.predict(x)
    img = annotate_image(config, x[0], y_pred[0], y[0])
    img = img.numpy()*255
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    cv2_imshow(img)